In [14]:
import os
import pandas as pd
import numpy as np
import json
import datetime
from pprint import pprint

In [3]:
course_axis = 'data/axis_DelftX_AE1110x_2T2015.csv'
event_log = 'data/DelftX_AE1110x_2T2015-events.log'
delft15dir = 'data/DelftX-AE1110x-2T2015/'

In [4]:
course_axisdf = pd.read_csv(course_axis)

In [8]:
with open(event_log,'r') as f:
    #pprint(json.loads(f.readline()))
    problem_check_data = []
    line_count = 0
    problem_check_count = 0
    fail_count = 0
    try:
        for line in f.readlines():
            line_content = json.loads(line)
            line_count+=1
#             if line_count%100000==0:
#                 print(line_content['event_type'])
            if line_content['event_type']=='problem_check':
                problem_check_count+=1
                problem_check_data.append(line_content)
                
    except:
        fail_count+=1
print(line_count)
print(problem_check_count)
print(fail_count)

play_video
seq_prev
pause_video
play_video
seek_video
problem_check
/courses/course-v1:DelftX+AE1110x+2T2015/discussion/forum/f0dca0e8ab9d4d649445113d106e4bd2/threads/55702ed638420615200000a0
pause_video
seek_video
play_video
/courses/course-v1:DelftX+AE1110x+2T2015/courseware/949231a21e6b407ca36ef54c34168452/
load_video
page_close
/courses/course-v1:DelftX+AE1110x+2T2015/progress
play_video
/courses/course-v1:DelftX+AE1110x+2T2015/courseware/9678d011194941d58db2390f583c5f81/
pause_video
seq_goto
play_video
pause_video
seq_goto
pause_video
seek_video
play_video
/courses/course-v1:DelftX+AE1110x+2T2015/courseware/669e9115be17498a9fa0cc5cee0cb78f/08a17b1b83234c8da3f3d4987cf86cf3/
seek_video
play_video
/courses/course-v1:DelftX+AE1110x+2T2015/courseware/669e9115be17498a9fa0cc5cee0cb78f/
problem_check
seq_prev
seq_goto
/courses/course-v1:DelftX+AE1110x+2T2015/courseware/9678d011194941d58db2390f583c5f81/38510976e5ee489eacc01e3107697029/
play_video
load_video
play_video
seek_video
pause_vide

In [9]:
def process_time(time):
    return datetime.datetime.strptime(time[:-6], '%Y-%m-%dT%H:%M:%S.%f' if '.' in time[:-6] else '%Y-%m-%dT%H:%M:%S')

In [33]:
protolist=[]
for entry in problem_check_data:
    temp_dict = {}
    temp_dict['username'] =  entry['username'].split('_')[1]
    temp_dict['time'] = process_time(entry['time'])
    temp_dict['problem_id'] = entry['event']['problem_id'].split("@")[-1]
    temp_dict["success"] = entry['event']['success']
    temp_dict["attempts"] = entry['event']['attempts']

    protolist.append(temp_dict)

In [31]:
# for x in problem_check_data:
#     if ("\\" in x['event']["problem_id"]):
#         print(x['event']["problem_id"])

In [35]:
problem_check_with_response_df = pd.DataFrame(protolist)

In [1]:
problem_check_with_response_df

NameError: name 'problem_check_with_response_df' is not defined

In [41]:
idx=0
problem_dict={}
for problem in problem_check_with_response_df.problem_id.unique():
    problem_dict[problem] = idx
    idx+=1

In [85]:
def process_groups(group,limit=1000):
    group = group.sort_values(by='time')
    indices=list(range(1,len(group["time"])+1))
    if len(group["time"]) > limit:
        indices[limit:] = [0]* (len(group["time"])-limit)
    group['timesteps'] = indices
    return group

In [87]:
grouped_userdf = problem_check_with_response_df.groupby('username').apply(process_groups)

In [89]:
grouped_userdf.head()

attempts                        problem_id  success  \
username                                                                
100663505 257108         1  e2ba3922c9374e089f30b068cd59eb85  correct   
          257109         1  a5e8909e66554554bd5517b8cf1fbb25  correct   
          255564         1  2d876aa861ae454589d96e80aa6e82fe  correct   
          255558         1  e45fa5e2a8e14f7cbdbe6e1a662a63c8  correct   
          255559         1  191a76b3412d4eb284583ab90ab16899  correct   

                                       time   username  timesteps  
username                                                           
100663505 257108 2015-09-17 20:15:19.860762  100663505          1  
          257109 2015-09-17 20:16:49.305596  100663505          2  
          255564 2015-09-17 20:22:07.486771  100663505          3  
          255558 2015-09-17 20:25:04.750014  100663505          4  
          255559 2015-09-17 20:26:25.969085  100663505          5

In [93]:
user_problem_sequence = grouped_userdf.pivot(index= 'username', columns = 'timesteps', values = 'problem_id')

In [102]:
grouped_userdf.pivot(index= 'username', columns = 'timesteps', values = 'success')

timesteps,1,2,3,4,5,6,7,8,9,10,...,836,837,838,839,840,841,842,843,844,845
username,,,,,,,,,,,,,,,,,,,,,
100663505,correct,correct,correct,correct,correct,correct,correct,incorrect,incorrect,incorrect,...,None,None,None,None,None,None,None,None,None,None
100664898,correct,incorrect,incorrect,incorrect,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
100666909,incorrect,correct,incorrect,incorrect,correct,incorrect,incorrect,incorrect,incorrect,incorrect,...,None,None,None,None,None,None,None,None,None,None
100668446,correct,incorrect,incorrect,correct,incorrect,incorrect,incorrect,correct,incorrect,incorrect,...,None,None,None,None,None,None,None,None,None,None
100670598,correct,correct,incorrect,incorrect,correct,incorrect,correct,incorrect,incorrect,incorrect,...,None,None,None,None,None,None,None,None,None,None
100671168,correct,incorrect,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
100673226,correct,incorrect,incorrect,incorrect,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
100677139,incorrect,correct,incorrect,incorrect,incorrect,incorrect,incorrect,incorrect,incorrect,incorrect,...,None,None,None,None,None,None,None,None,None,None
100679240,incorrect,correct,incorrect,incorrect,correct,incorrect,correct,correct,correct,incorrect,...,None,None,None,None,None,None,None,None,None,None


In [101]:
grouped_userdf

attempts                        problem_id    success  \
username                                                                  
100663505 257108         1  e2ba3922c9374e089f30b068cd59eb85    correct   
          257109         1  a5e8909e66554554bd5517b8cf1fbb25    correct   
          255564         1  2d876aa861ae454589d96e80aa6e82fe    correct   
          255558         1  e45fa5e2a8e14f7cbdbe6e1a662a63c8    correct   
          255559         1  191a76b3412d4eb284583ab90ab16899    correct   
          255560         2  191a76b3412d4eb284583ab90ab16899    correct   
          255565         1  6636f73d28a34b43842af563268d3ca6    correct   
          255563         1  654330c5876d4857bc1c326e61780db5  incorrect   
          255557         1  106e84ede5c94f5f8c91d3d5e1c2a11c  incorrect   
          255566         1  6faec79f96404f12bc6b4b225db8ecad  incorrect   
          255556         1  f52672dff65e46959160e268d384b161    correct   
          255562         1  4b6002cf857c4aa6a9792ea35200e853    correct   
          255561         2  106e84ede5c94f5f8c91d3d5e1c2a11c    correct   
          257658         1  b8ca0effd8db42dba28d14a90cbc1163    correct   
          257623         1  27b146ac218f4c35bf48a889dcef69cf    correct   
          257611         1  fccea325fd314fe4ab1aed30138a65ad  incorrect   
          257612         2  fccea325fd314fe4ab1aed30138a65ad  incorrect   
          257613         3  fccea325fd314fe4ab1aed30138a65ad    correct   
          257604         1  7453da62ea5f4a1e9d5ed0ccf326f687    correct   
          258665         1  b95a271e99754f4782537f05b88da5bd  incorrect   
          258664         2  b95a271e99754f4782537f05b88da5bd  incorrect   
          258663         3  b95a271e99754f4782537f05b88da5bd  incorrect   
          258662         1  d56e255937bf4e1a9d59bbe754f735dd  incorrect   
100664898 49196          1  b8ca0effd8db42dba28d14a90cbc1163    correct   
          49197          1  5a06f0135c9c46a1843767519726753c  incorrect   
          49198          2  5a06f0135c9c46a1843767519726753c  incorrect   
          49199          3  5a06f0135c9c46a1843767519726753c  incorrect   
100666909 71380          1  b8ca0effd8db42dba28d14a90cbc1163  incorrect   
          71376          2  b8ca0effd8db42dba28d14a90cbc1163    correct   
          71377          1  5a06f0135c9c46a1843767519726753c  incorrect   
...                    ...                               ...        ...   
9662592   355327         3  fb6aa70a3aef40db8721a3466c79a64d  incorrect   
          355328         1  c75e19ca14fa4928aa5e0542eb9ae8e7  incorrect   
          355329         2  c75e19ca14fa4928aa5e0542eb9ae8e7  incorrect   
          355330         1  96375285b2db45d1829b161457a6354e  incorrect   
          355331         2  96375285b2db45d1829b161457a6354e  incorrect   
          355332         1  aa21b57742a44f7b8f8678b58e7ae647  incorrect   
          355333         2  aa21b57742a44f7b8f8678b58e7ae647  incorrect   
          355334         1  106f24b51a5440d58da99918a614e419  incorrect   
          355335         2  106f24b51a5440d58da99918a614e419  incorrect   
          355336         1  7fa9008c65df45b2a5adfbf2719c3b53  incorrect   
          355337         2  7fa9008c65df45b2a5adfbf2719c3b53  incorrect   
          355338         3  7fa9008c65df45b2a5adfbf2719c3b53  incorrect   
          355339         1  25eadfc7d12d4919b7729b20a8203c3a  incorrect   
          355340         2  25eadfc7d12d4919b7729b20a8203c3a  incorrect   
          355341         3  25eadfc7d12d4919b7729b20a8203c3a  incorrect   
          355342         1  9875a69ca03f4925b51b856c3d5f55d0  incorrect   
          355343         2  9875a69ca03f4925b51b856c3d5f55d0  incorrect   
          355344         3  9875a69ca03f4925b51b856c3d5f55d0  incorrect   
          355345         1  72346928d69247b2bab1845e294646fa  incorrect   
          355346         2  72346928d69247b2bab1845e294646fa  incorrect   
          355347         3  72346928d69247b